In [1]:
import torch
import resiliparse
from resiliparse import parse
from cs336_data.utils import extract_text_from_html_bytes

from warcio.archiveiterator import ArchiveIterator
import gzip
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding


In [2]:
warc_file_path = '../data/CC-MAIN-20180420081400-20180420101400-00000.warc.gz'
wet_file_path = '../data/CC-MAIN-20180420081400-20180420101400-00000.warc.wet.gz'

In [9]:
def load_html_from_warc(warc_path, doc_index=0):
    """
    Load HTML content from a WARC file by document index.

    Args:
        warc_path (str): Path to the .warc.gz file.
        doc_index (int): Index of the document to load (0-based).

    Returns:
        tuple: (raw_html_bytes, url) or (None, None) if index not found.
    """
    current_idx = -1  # Counter for response records
    with gzip.open(warc_path, 'rb') as stream:
        for record in ArchiveIterator(stream):
            # Only consider response records (actual HTTP responses containing HTML)
            if record.rec_type == 'response':
                current_idx += 1
                if current_idx == doc_index:
                    raw_html = record.content_stream().read()
                    url = record.rec_headers.get_header('WARC-Target-URI')
                    return raw_html, url
    return None, None




In [ ]:
def load_text_from_wet(wet_path, doc_index=0):
    """
    Load plain text content from a WET file by document index.

    Args:
        wet_path (str): Path to the .wet.gz file.
        doc_index (int): Index of the document to load (0-based).

    Returns:
        str: Plain text of the selected document or empty string if not found.
    """
    current_idx = -1
    content = []
    is_target_doc = False

    with gzip.open(wet_path, 'rt', encoding='utf-8') as file:
        for line in file:
            if line.startswith('WARC/1.0'):
                if is_target_doc:
                    # End of the desired document
                    break
                current_idx += 1
                content = []
                is_target_doc = False
                continue

            # WET header and content are separated by an empty line
            if line.strip() == '' and not is_target_doc:
                if current_idx == doc_index:
                    is_target_doc = True  # Next lines belong to target document
                continue

            if is_target_doc:
                content.append(line.rstrip())

    return '\n'.join(content).strip()

In [11]:
# Load and decode HTML content from WARC file
html_bytes, url = load_html_from_warc(warc_file_path, doc_index=2)

if html_bytes:
    try:
        html_str = html_bytes.decode('utf-8')
    except UnicodeDecodeError:
        detected_encoding = detect_encoding(html_bytes)
        html_str = html_bytes.decode(detected_encoding, errors='replace')

    print("URL:", url)
    print("HTML content (first 1000 characters):")
    print(html_str[:1000])
else:
    print("No HTML content found in WARC file.")

# Load plain text content from WET file
wet_text = load_text_from_wet(wet_file_path, doc_index=2)

print("\nPlain text from WET file (first 1000 characters):")
print(wet_text[:1000])


URL: http://00secretsweety.ozforex.info/pissysexy21-video
HTML content (first 2000 characters):
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><title>pissysexy21    daddysspankk videos</title><link href="http://fonts.googleapis.com/css?family=Arvo:400,700" rel="stylesheet" type="text/css" /><style media="all" /> 
html,body{height:98%}body{background:#cc0000 ;font-family:Arvo, serif;font-size:12px;color:#545454;margin:0;padding:0}h1,h2,h3{text-transform:uppercase;font-weight:300;font-family:Arvo, serif;margin:0;padding:0}h2{font-size:1.5em;padding:0 0 20px}p,ol,ul{margin-top:0}p{line-height:180%}a:hover{text-decoration:none}a img{border:none}img.alignright{float:right}img.aligncenter{margin:0 auto}
hr{display:none}#wrapper{overflow:hidden;width:1200px;background:#FFF;box-shadow:0 0 10px 5px rgba(0,0,0,.2);marg

In [12]:
print("HTML content:")
print(html_str)

HTML content:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><title>pissysexy21    daddysspankk videos</title><link href="http://fonts.googleapis.com/css?family=Arvo:400,700" rel="stylesheet" type="text/css" /><style media="all" /> 
html,body{height:98%}body{background:#cc0000 ;font-family:Arvo, serif;font-size:12px;color:#545454;margin:0;padding:0}h1,h2,h3{text-transform:uppercase;font-weight:300;font-family:Arvo, serif;margin:0;padding:0}h2{font-size:1.5em;padding:0 0 20px}p,ol,ul{margin-top:0}p{line-height:180%}a:hover{text-decoration:none}a img{border:none}img.alignright{float:right}img.aligncenter{margin:0 auto}
hr{display:none}#wrapper{overflow:hidden;width:1200px;background:#FFF;box-shadow:0 0 10px 5px rgba(0,0,0,.2);margin:50px auto}.container{width:1200px;margin:0 auto}.clearfix{clear:both}#header{ov

In [13]:
print("\nPlain text from WET file:")
print(wet_text)


Plain text from WET file:
ウィークリーマンション、マンスリーマンション高知の00マンスリードットコム
ウィークリーマンション、マンスリーマンション 高知
ウィークリー、マンスリーマンションで
敷金0礼金ゼロで家具付き賃貸マンションで仮住まい
全国のウィークリーマンション・マンスリーマンション検索サイト
敷金0・礼金0・仲介手数料0のマンスリーマンション。ウィークリーマンション情報
サイトマップ
お気に入りに追加
TOP
初めての方へ
マンスリーマンション活用術
よくある質問
お客様質問箱
会社概要
高知TOP
お客様質問箱
００マンスリーでは、さらなるサービス品質向上の為、お客様からのご意見・ご要望をお待ちしております。
ご意見地域 北海道 青森県 岩手県 宮城県 秋田県 山形県 福島県 茨城県 栃木県 群馬県 埼玉県 千葉県 東京都 神奈川県 新潟県 富山県 石川県 福井県 山梨県 長野県 岐阜県 静岡県 愛知県 三重県 滋賀県 京都府 大阪府 兵庫県 奈良県 和歌山県 鳥取県 島根県 岡山県 広島県 山口県 徳島県 香川県 愛媛県 高知県 福岡県 佐賀県 長崎県 熊本県 大分県 宮崎県 鹿児島県 沖縄県
お問合せ内容 マンションについて 料金について 契約について 物件について サイトについて その他
メッセージ
お電話からでも条件ピッタリのおすすめウィークリー・マンスリー物件が必ず見つかります！
マンスリーマンション、
ウィークリーマンション、
家具付き賃貸、
全国マンスリー物件、
レンタカー
敷金礼金0、
リゾート、
動画、
空室対策、
東京、
宮城、
福岡、
沖縄
ウィークリー、マンスリーマンションで
敷金0礼金ゼロで家具付き賃貸マンションで仮住まい
敷金0礼金ゼロ家具付き賃貸！仮住まい、短期、長期、単身赴任、研修などにピッタリ！
ホーム
会社概要
プライバシーポリシー
サイトマップ
Copyright (C) 2009 Business Search All Right Reserved.
